## JAZMIN VIVIANA SANTOYO VEGA
###**Sistema de Comunicación de un Robot Espacial**
####**Teoría de la Información **

In [1]:
import random
import pickle
import math
import matplotlib.pyplot as plt
from collections import Counter
import heapq
from collections import defaultdict
import pandas as pd
import struct

###Fuente de Información

In [2]:
# Aquí la fuente de información son los sensores del robot recopilan datos
def recopilar_datos():
  # Simulacion de algunos atributos que recopila el sensor del robot
  objetos = ["mineral, roca & meteorito", "roca, cristal, agua & hielo", "cristal", "hielo, relieves, regolito & meteorito", "agua, meterorito & relieves","objeto no identificado"]
  #objetos = ["agua"]
  ubicacion = (round(random.uniform(-1000, 1000), 2), round(random.uniform(-1000, 1000), 2), round(random.uniform(-1000, 1000), 2))
  imagenes = ["imagen1.jpg", "imagen2.jpg"]

  objeto = random.choice(objetos)
  datos_recopilados = {
      "Se detectó": objeto,
      "Ubicación": ubicacion,
      "Imagenes": imagenes
  }
  return datos_recopilados

 ### Calcular frecuencia

In [3]:
# recibe una cadena de texto o de numeros para calcular la frecuencia y devolver a la codificación que se desee
def obtiene_frecuencia(cadena):
    frecuencias = dict(Counter(cadena))
    return frecuencias

### Codificación Binaria

In [4]:
def codificacion_binaria(cadena):
    # codifica la cadena en binario
    cadena_codificada_binario = ''.join(format(ord(c), '08b') for c in cadena)

    #  diccionario de códigos para cada símbolo
    codigos_binario = {c: format(ord(c), '08b') for c in cadena}

    return cadena_codificada_binario, codigos_binario

### Codificación Huffman

In [5]:
def crea_arbol_huffman(frecuencias):
  # lista de nodos, donde cada nodo es [frecuencia, [numero, codigo]]
    heap = [[frecuencia, [numero, ""]] for numero, frecuencia in frecuencias.items()]
    # convierto la lista en una cola de prioridad
    heapq.heapify(heap)
    # mientras haya más de un nodo en la cola de prioridad (árbol incompleto)
    while len(heap) > 1:
        menos_frecuente = heapq.heappop(heap)
        mas_frecuente = heapq.heappop(heap)

        for elem in menos_frecuente[1:]:
            elem[1] = '0' + elem[1]

        for elem in mas_frecuente[1:]:
            elem[1] = '1' + elem[1]
            # combino los dos nodos en uno y lo vuelve a insertar en la cola de prioridad
        heapq.heappush(heap, [menos_frecuente[0] + mas_frecuente[0]] + menos_frecuente[1:] + mas_frecuente[1:])

    return sorted(heapq.heappop(heap)[1:], key=lambda p: (len(p[-1]), p))

In [6]:
def codificacion_huffman(cadena):
  # llamo a mi función que obtiene la frecuencia y a la que crea el árbol huffman
    frecuencia_simbolo = obtiene_frecuencia(cadena)
    arbol_huffman = crea_arbol_huffman(frecuencia_simbolo)

    codigos_huffman = {}
    for numero, codigo in arbol_huffman:
        codigos_huffman[numero] = codigo
    # la cadena codificada
    cadena_codificada_huffman = ''.join([codigos_huffman[numero] for numero in cadena])

    return cadena_codificada_huffman, codigos_huffman

### Codificación Shannon-Fano

In [7]:
def codificacion_shannon_fano(cadena):
    def shannon_fano_recursivo(frecuencias):
        if len(frecuencias) == 1:
            simbolo = list(frecuencias.keys())[0]
            return {simbolo: ''}
        # Paso 1: Ordeno símbolos por frecuencia (mayor a menor)
        frecuencias_ordenadas = sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)

        # Paso 2: Divido la lista en dos partes
        mitad = len(frecuencias_ordenadas) // 2
        mitad_izquierda = frecuencias_ordenadas[:mitad]
        mitad_derecha = frecuencias_ordenadas[mitad:]

        # Paso 3: Asigno dígitos binarios 0 y 1
        codigos_sf = {}
        for simbolo, freq in mitad_izquierda:
            codigos_sf[simbolo] = '0'
        for simbolo, freq in mitad_derecha:
            codigos_sf[simbolo] = '1'

        # Paso 4: Aplico recursivamente a cada mitad
        codigos_izquierda = shannon_fano_recursivo({k: v for k, v in mitad_izquierda})
        for simbolo, codigo in codigos_izquierda.items():
            codigos_sf[simbolo] = '0' + codigo
        codigos_derecha = shannon_fano_recursivo({k: v for k, v in mitad_derecha})
        for simbolo, codigo in codigos_derecha.items():
            codigos_sf[simbolo] = '1' + codigo

        return codigos_sf

    frecuencias = obtiene_frecuencia(cadena)
    codigos_sf = shannon_fano_recursivo(frecuencias)

    # Codifico la cadena
    cadena_codificada_sf = ''.join(codigos_sf[simbolo] for simbolo in cadena)

    return cadena_codificada_sf, codigos_sf

### Codificación Gray

In [8]:
def codificacion_gray(cadena):
    def gray_codigo(num):
        return num ^ (num >> 1)

    # obtengo la longitud de la cadena
    longitud = len(cadena)

    # convierto la cadena de entrada en una secuencia de bits
    bits = ''.join(format(ord(char), '08b') for char in cadena)

    # diccionario para asignar códigos de Gray a cada símbolo
    codigos_gray = {}

    for i in range(longitud):
        simbolo = cadena[i]
        num = int(bits[i*8:(i+1)*8], 2)
        codigo = gray_codigo(num)
        codigos_gray[simbolo] = format(codigo, f'0{8}b')

    # codifico la cadena de entrada utilizando los códigos de Gray
    cadena_codificada_gray = "".join([codigos_gray[s] for s in cadena])

    return cadena_codificada_gray, codigos_gray

### Codificación Tunstall

In [9]:
def codificacion_tunstall(cadena):

    frecuencias = obtiene_frecuencia(cadena)

    # creo una lista de tuplas (símbolo, frecuencia) ordenadas por frecuencia
    simbolos_frecuencias = sorted(frecuencias.items(), key=lambda x: x[1], reverse=True)

    # diccionario para asignar códigos Tunstall a cada símbolo
    codigos_tunstall = {}
    num_bits = 0

    while len(simbolos_frecuencias) > 2 ** num_bits:
        num_bits += 1

    for i, (simbolo, _) in enumerate(simbolos_frecuencias):
        codigo = format(i, f'0{num_bits}b')
        codigos_tunstall[simbolo] = codigo

    # la cadena de entrada utilizando los códigos Tunstall
    cadena_codificada_t = "".join([codigos_tunstall[s] for s in cadena])

    return cadena_codificada_t, codigos_tunstall

### Transmisor con menu

In [10]:
def transmisor_de_la_nave_espacial(datos_recopilados):
    objeto_detectado = datos_recopilados["Se detectó"]
    ubicacion = datos_recopilados["Ubicación"]
    imagenes = datos_recopilados["Imagenes"]

    print("Elige el tipo de codificación:")
    print("1. Binario")
    print("2. Huffman")
    print("3. Shannon-Fano")
    print("4. Gray")
    print("5. Tunstall")
    opcion = input("Ingrese el número correspondiente a la opción deseada: ")

    if opcion == "1":
        # CODIFICACION BINARIO
        cadena_objeto_codificada, codigos_objeto = codificacion_binaria(objeto_detectado)
        cadena_ubicacion_codificada, codigos_ubicacion = codificacion_binaria(str(ubicacion))
        df_codigos_objeto = pd.DataFrame(list(codigos_objeto.items()), columns=['Simbolo', 'Código'])
        df_codigos_ubicacion = pd.DataFrame(list(codigos_ubicacion.items()), columns=['Simbolo', 'Código'])

        print("Handshake de objeto en binario:")
        print(df_codigos_objeto)
        print("\nHandshake de ubicación en binario:")
        print(df_codigos_ubicacion)
      # los datos codificados con Huffman

        datos_transmitidos = {
            "Se detectó": cadena_objeto_codificada,
            "Ubicación": cadena_ubicacion_codificada,
            "Imagenes": imagenes
        }

        print("Datos codificados en Huffman:")
        print(datos_transmitidos)

    elif opcion == "2":
        # CODIFICACION HUFFMAN
        cadena_objeto_codificada, codigos_objeto = codificacion_huffman(objeto_detectado)
        cadena_ubicacion_codificada, codigos_ubicacion = codificacion_huffman(str(ubicacion))
        df_codigos_objeto = pd.DataFrame(list(codigos_objeto.items()), columns=['Simbolo', 'Código'])
        df_codigos_ubicacion = pd.DataFrame(list(codigos_ubicacion.items()), columns=['Simbolo', 'Código'])

        print("Handshake de objeto en Huffman:")
        print(df_codigos_objeto)
        print("\nHandshake de ubicación en Huffman:")
        print(df_codigos_ubicacion)
      # los datos codificados con Huffman

        datos_transmitidos = {
            "Se detectó": cadena_objeto_codificada,
            "Ubicación": cadena_ubicacion_codificada,
            "Imagenes": imagenes
        }

        print("Datos codificados en Huffman:")
        print(datos_transmitidos)

    elif opcion == "3":

      # CODIFICACIÓN SHANNON FANO
      cadena_objeto_codificada, codigos_objeto = codificacion_shannon_fano(objeto_detectado)
      cadena_ubicacion_codificada, codigos_ubicacion = codificacion_shannon_fano(str(ubicacion))
      df_codigos_objeto = pd.DataFrame(list(codigos_objeto.items()), columns=['Simbolo', 'Código'])
      df_codigos_ubicacion = pd.DataFrame(list(codigos_ubicacion.items()), columns=['Simbolo', 'Código'])
      print("Handshake de objeto en Shannon-Fano:")
      print(df_codigos_objeto)
      print("\nHandshake de ubicación en Shannon-Fano")
      print(df_codigos_ubicacion)

      datos_transmitidos = {
          "Se detectó": cadena_objeto_codificada,
          "Ubicación": cadena_ubicacion_codificada,
          "Imagenes": imagenes
      }
      print("Datos codificados en Shannon-Fano:")
      print(datos_transmitidos)

    elif opcion == "4":
      # CODIFICACIÓN GRAY
      cadena_objeto_codificada, codigos_objeto = codificacion_gray(objeto_detectado)
      cadena_ubicacion_codificada, codigos_ubicacion = codificacion_gray(str(ubicacion))
      df_codigos_objeto = pd.DataFrame(list(codigos_objeto.items()), columns=['Simbolo', 'Código'])
      df_codigos_ubicacion = pd.DataFrame(list(codigos_ubicacion.items()), columns=['Simbolo', 'Código'])
      print("Handshake de objeto en Gray:")
      print(df_codigos_objeto)
      print("\nHandshake de ubicación en Gray")
      print(df_codigos_ubicacion)

      datos_transmitidos = {
          "Se detectó": cadena_objeto_codificada,
          "Ubicación": cadena_ubicacion_codificada,
          "Imagenes": imagenes
      }
      print("Datos codificados en Gray:")
      print(datos_transmitidos)

    elif opcion == "5":
      # CODIFICACIÓN TUNSTALL
      cadena_objeto_codificada, codigos_objeto = codificacion_tunstall(objeto_detectado)
      cadena_ubicacion_codificada, codigos_ubicacion = codificacion_tunstall(str(ubicacion))
      df_codigos_objeto = pd.DataFrame(list(codigos_objeto.items()), columns=['Simbolo', 'Código'])
      df_codigos_ubicacion = pd.DataFrame(list(codigos_ubicacion.items()), columns=['Simbolo', 'Código'])
      print("Handshake de objeto en Tunstall:")
      print(df_codigos_objeto)
      print("\nHandsdhake de ubicación en Tunstall")
      print(df_codigos_ubicacion)

      datos_transmitidos = {
          "Se detectó": cadena_objeto_codificada,
          "Ubicación": cadena_ubicacion_codificada,
          "Imagenes": imagenes
      }
      print("Datos codificados en Tunstall:")
      print(datos_transmitidos)

    else:
        print("Opción no válida. Debe ingresar 1,  2, 3, 4 o 5.")

    return datos_transmitidos, codigos_objeto, codigos_ubicacion

###Canal

In [11]:
# Este es el medio por el que viajaran los datos transmitidos
# esta función simula el canal, en este caso es por medio de fibra óptica
def fibra_optica(datos_transmitidos):

    """ DETALLES DE LA FIBRA ÓPTICA """
    # longitud de la fibra en km
    longitud_km = 15
    # velocidad de la luz en ms
    velocidad_ms = 7000
    # longitud de la fibra a metros
    longitud_m = longitud_km * 1000
    # tiempo de propagación en segundos
    tiempo = longitud_m / velocidad_ms

    """ CODIFICACIÓN ADAPTATIVA"""
    """ PAQUETES"""
    objeto_codificado = datos_transmitidos["Se detectó"]
    ubicacion_codificada = datos_transmitidos["Ubicación"]
    imagenes_codificada = datos_transmitidos["Imagenes"]
    datos_canal = datos_transmitidos.copy()

    # divido la cadena del objeto en 4 paquetes
    paquete1_objeto = objeto_codificado[:len(objeto_codificado) // 4]
    paquete2_objeto = objeto_codificado[len(objeto_codificado) // 4: 2 * (len(objeto_codificado) // 4)]
    paquete3_objeto = objeto_codificado[2 * (len(objeto_codificado) // 4): 3 * (len(objeto_codificado) // 4)]
    paquete4_objeto = objeto_codificado[3 * (len(objeto_codificado) // 4):]

    # divido la cadena de ubicación en 4 paquetes
    paquete1_ubicacion = ubicacion_codificada[:len(ubicacion_codificada) // 4]
    paquete2_ubicacion = ubicacion_codificada[len(ubicacion_codificada) // 4: 2 * (len(ubicacion_codificada) // 4)]
    paquete3_ubicacion = ubicacion_codificada[2 * (len(ubicacion_codificada) // 4): 3 * (len(ubicacion_codificada) // 4)]
    paquete4_ubicacion = ubicacion_codificada[3 * (len(ubicacion_codificada) // 4):]

    # visualizo los paquetes del objeto
    print("Paquetes del objeto:")
    print("paquete1_objeto:", paquete1_objeto)
    print("paquete2_objeto:", paquete2_objeto)
    print("paquete3_objeto:", paquete3_objeto)
    print("paquete4_objeto:", paquete4_objeto)

    # visualizo los paquetes de ubicación
    print("Paquetes de ubicación:")
    print("paquete1_ubicacion:", paquete1_ubicacion)
    print("paquete2_ubicacion:", paquete2_ubicacion)
    print("paquete3_ubicacion:", paquete3_ubicacion)
    print("paquete4_ubicacion:", paquete4_ubicacion)

    """CANALES"""

    canal1 = []  # Canal 1
    canal2 = []  # Canal 2
    canal3 = []  # Canal 3
    canal4 = []  # Canal 4
    canal5 = []  # Canal 5

    # lista de canales
    canales = [canal1, canal2, canal3, canal4, canal5]

    # lista de paquetes
    paquetes = [paquete1_objeto, paquete2_objeto, paquete3_objeto, paquete4_objeto, paquete1_ubicacion, paquete2_ubicacion, paquete3_ubicacion, paquete4_ubicacion]

    paquetes_recibidos_objeto = []  # para paquetes completos de objetos
    paquetes_recibidos_ubicacion = []  # para paquetes completos de ubicación

    # INICIO LA CODIFICACIÓN ADAPTATIVA DE MIS CANALES

    # 1.- Mientras haya paquetes en la lista
    while paquetes:
        # 2.- Se elige un canal aleatoriamente de la lista de canales
        canal_elegido = random.choice(canales)
        # 3.- Se genera un número aleatorio entre 1 y 5 para determinar cuántos paquetes se seleccionarán
        num_elementos_aleatorios = random.randint(1, min(len(paquetes), 5))
        # 4.- Se seleccionan elementos aleatorios de la lista 'paquetes' en la cantidad determinada anteriormente
        elementos_seleccionados = random.sample(paquetes, num_elementos_aleatorios)
        # 5.- Al canal elegido se van añadiendo los elementos seleccionados
        canal_elegido.extend(elementos_seleccionados)
        # 6.- Guardo en una lista de elementos no seleccionados que quedan en 'paquetes'
        elementos_no_seleccionados = [elemento for elemento in paquetes if elemento not in elementos_seleccionados]
        # 7.- Actualizo la lista 'paquetes' con los elementos no seleccionados
        paquetes = elementos_no_seleccionados

        # 8.- Añado paquetes completos a las variables correspondientes
        for elemento in elementos_seleccionados:
            if elemento in [paquete1_objeto, paquete2_objeto, paquete3_objeto, paquete4_objeto]:
                paquetes_recibidos_objeto.append(elemento)
            else:
                paquetes_recibidos_ubicacion.append(elemento)

        # Visualizo el canal y los paquetes que ingresaron a ese canal
        print("Canal:", canales.index(canal_elegido)+1)

        print("Paquetes recibidos:")
        for elemento in canal_elegido:
            print(elemento)

        if elementos_no_seleccionados: #Imprime "Paquete no recibidos:" solo cuando hay paquetes no seleccionados
          print("Paquete no recibidos:")
          for elemento in elementos_no_seleccionados:
            print(elemento)

        print()

    # imprimo para asegurar que todos los paquetes fueron recibidos
    print("paquetes recibidos objeto: ", paquetes_recibidos_objeto)
    print("paquetes recibidos ubicacion: ", paquetes_recibidos_ubicacion)

    # ordenos por orden mis paquetes para recuperar el mensaje original
    paquetes_recibidos_objeto.sort(key=lambda x: [paquete1_objeto, paquete2_objeto, paquete3_objeto, paquete4_objeto].index(x))
    paquetes_recibidos_ubicacion.sort(key=lambda x: [paquete1_ubicacion, paquete2_ubicacion, paquete3_ubicacion, paquete4_ubicacion].index(x))

    # uno los paquetes y actualizo el diccionario datos_canal
    objeto_codificado_nuevo = ''.join(paquetes_recibidos_objeto)
    ubicacion_codificada_nueva = ''.join(paquetes_recibidos_ubicacion)
    datos_canal["Se detectó"] = objeto_codificado_nuevo
    datos_canal["Ubicación"] = ubicacion_codificada_nueva

    print("Paquetes recibidos objeto ordenados:", paquetes_recibidos_objeto)
    print("Paquetes recibidos ubicación ordenados:", paquetes_recibidos_ubicacion)

    return datos_canal, tiempo

#### Decodificación Binario

In [12]:
def decodificacion_binaria(cadena_codificada_binario, codigos_binario):
    cadena_binaria_decodificada = ""
    codigo_actual = ""

    for bit in cadena_codificada_binario:
        codigo_actual += bit
        for simbolo, codigo in codigos_binario.items():
            if codigo_actual == codigo:
                cadena_binaria_decodificada += simbolo
                codigo_actual = ""
                break

    return cadena_binaria_decodificada

#### Decodificación Huffman

In [13]:
def decodificacion_huffman(cadena_codificada_huffman, codigos_huffman):
    cadena_decodificada = ""
    codigo_actual = ""
    for bit in cadena_codificada_huffman:
        codigo_actual += bit
        for simbolo, codigo in codigos_huffman.items():
            if codigo == codigo_actual:
                cadena_decodificada += simbolo
                codigo_actual = ""
                break
    return cadena_decodificada

#### Decodificación Shannon-Fano

In [14]:
def decodificacion_shannon_fano(cadena_codificada_sf, codigos_sh):
    cadena_decodificada = ""
    codigo_actual = ""

    for bit in cadena_codificada_sf:
        codigo_actual += bit
        for simbolo, codigo in codigos_sh.items():
            if codigo_actual == codigo:
                cadena_decodificada += simbolo
                codigo_actual = ""
                break

    return cadena_decodificada

#### Decodificación Gray

In [15]:
def decodificacion_gray(cadena_codificada_gray, codigos_gray):
    def reverse_gray_code(gray_code):
        num = gray_code
        while gray_code >= 1:
            gray_code >>= 1
            num ^= gray_code
        return num

    # invierto el diccionario de codificación Gray para buscar códigos en lugar de símbolos
    codigos_invertidos = {codigo: simbolo for simbolo, codigo in codigos_gray.items()}

    # decodifico la cadena codificada en Gray
    cadena_gray_decodificada = ""
    codigo_actual = ""

    for bit in cadena_codificada_gray:
        codigo_actual += bit
        if codigo_actual in codigos_invertidos:
            simbolo = codigos_invertidos[codigo_actual]
            cadena_gray_decodificada += simbolo
            codigo_actual = ""

    return cadena_gray_decodificada

#### Decodificación Tunstall

In [16]:
def decodificacion_tunstall(cadena_codificada_t, codigos_tunstall):
    cadena_tunstall_decodificada = ""
    codigo_actual = ""

    for bit in cadena_codificada_t:
        codigo_actual += bit
        simbolo_encontrado = None

        for simbolo, codigo in codigos_tunstall.items():
            if codigo_actual.startswith(codigo):
                simbolo_encontrado = simbolo
                break

        if simbolo_encontrado is not None:
            cadena_tunstall_decodificada += simbolo_encontrado
            codigo_actual = codigo_actual[len(codigo):]

    return cadena_tunstall_decodificada

### Receptor

In [17]:
def receptor(datos_con_ruido, codigos_objeto, codigos_ubicacion):
    print("Dependiendo del método de codificación que elegiste,")
    print("elige el método de decodificación:")
    print("1. Binario")
    print("2. Huffman")
    print("3. Shannon-Fano")
    print("4. Gray")
    print("5. Tunstall")
    opcion = input("Ingrese el número correspondiente a la opción deseada: ")
    # PARE EL RUIDO ASI QUE NO ESTAN SALIENDO CON RUIDO

    objeto_codificado = datos_con_ruido["Se detectó"]
    ubicacion_codificada= datos_con_ruido["Ubicación"]
    imagenes = datos_con_ruido["Imagenes"]

    if opcion == "1":
        # Decodificar binario
        objeto_decodificado = decodificacion_binaria(objeto_codificado, codigos_objeto)
        ubicacion_decodificada = decodificacion_binaria(ubicacion_codificada, codigos_ubicacion)

    elif opcion == "2":
        # Decodificar Huffman
        objeto_decodificado = decodificacion_huffman(objeto_codificado, codigos_objeto)
        cadena_ubicacion = decodificacion_huffman(ubicacion_codificada, codigos_ubicacion)
        ubicacion_decodificada = tuple(map(float, cadena_ubicacion.strip("()").split(", ")))

    elif opcion == "3":
        # Decodificar Shannon-Fano
        objeto_decodificado = decodificacion_shannon_fano(objeto_codificado, codigos_objeto)
        ubicacion_decodificada = decodificacion_shannon_fano(ubicacion_codificada, codigos_ubicacion)
    elif opcion == "4":
      # Decodificar Gray
        objeto_decodificado = decodificacion_gray(objeto_codificado, codigos_objeto)
        ubicacion_decodificada = decodificacion_gray(ubicacion_codificada, codigos_ubicacion)
    elif opcion == "5":
        objeto_decodificado = decodificacion_tunstall(objeto_codificado, codigos_objeto)
        ubicacion_decodificada = decodificacion_tunstall(ubicacion_codificada, codigos_ubicacion)
    else:
        print("Opción no válida. Debe ingresar 1, 2, 3, 4 o 5.")
        return

    datos_decodificados = {
        "Se detectó": objeto_decodificado,
        "Ubicación": ubicacion_decodificada,
        "Imagenes": imagenes
    }

    return datos_decodificados

#### Simulamos que la información ha llegado a la Estación Espacial

### Destino de Información

Recopilo los datos

In [18]:
info_original = recopilar_datos()

Visualización del Handshake y la cadena codifcada en la codificaión que eligió el usuario

In [19]:
# Guardo las funciones en variables
info_transmitida, codigos_objeto, codigos_ubicacion = transmisor_de_la_nave_espacial(info_original)

Elige el tipo de codificación:
1. Binario
2. Huffman
3. Shannon-Fano
4. Gray
5. Tunstall
Ingrese el número correspondiente a la opción deseada: 4
Handshake de objeto en Gray:
   Simbolo    Código
0        r  01001011
1        o  01011000
2        c  01010010
3        a  01010001
4        ,  00111010
5           00110000
6        i  01011101
7        s  01001010
8        t  01001110
9        l  01011010
10       g  01010100
11       u  01001111
12       &  00110101
13       h  01011100
14       e  01010111

Handshake de ubicación en Gray
   Simbolo    Código
0        (  00111100
1        -  00111011
2        8  00100100
3        7  00101100
4        .  00111001
5        6  00101101
6        5  00101111
7        ,  00111010
8           00110000
9        2  00101011
10       1  00101001
11       )  00111101
Datos codificados en Gray:
{'Se detectó': '010010110101100001010010010100010011101000110000010100100100101101011101010010100100111001010001010110100011101000110000010100010101010001001

Visualización del la codificación adapatativa

In [20]:
datos_canal, tiempo_propagacions = fibra_optica(info_transmitida)

Paquetes del objeto:
paquete1_objeto: 010010110101100001010010010100010011101000110000010100
paquete2_objeto: 100100101101011101010010100100111001010001010110100011
paquete3_objeto: 101000110000010100010101010001001111010100010011000000
paquete4_objeto: 110101001100000101110001011101010101110101101001011000
Paquetes de ubicación:
paquete1_ubicacion: 00111100001110110010010000101100001110010010110100
paquete2_ubicacion: 10111100111010001100000010110000101101001010110011
paquete3_ubicacion: 10010010111100101101001110100011000000111011001011
paquete4_ubicacion: 11001010010010100100111001001011110010111100111101
Canal: 2
Paquetes recibidos:
11001010010010100100111001001011110010111100111101
110101001100000101110001011101010101110101101001011000
101000110000010100010101010001001111010100010011000000
010010110101100001010010010100010011101000110000010100
00111100001110110010010000101100001110010010110100
Paquete no recibidos:
100100101101011101010010100100111001010001010110100011
10111100111

Menu de decodificación

In [21]:
info_receptada = receptor(datos_canal, codigos_objeto, codigos_ubicacion)

Dependiendo del método de codificación que elegiste,
elige el método de decodificación:
1. Binario
2. Huffman
3. Shannon-Fano
4. Gray
5. Tunstall
Ingrese el número correspondiente a la opción deseada: 4


In [22]:
# Finalmente la información ha llegado a los científicos de la Estación Espacial
print("===== Datos Recibidos en la Estación Espacial =====")
print("===== Perseverance 2.1 ha capturado los siguientes datos ===== \n")
print(info_receptada)

===== Datos Recibidos en la Estación Espacial =====
===== Perseverance 2.1 ha capturado los siguientes datos ===== 

{'Se detectó': 'roca, cristal, agua & hielo', 'Ubicación': '(-87.65, 762.56, -511.55)', 'Imagenes': ['imagen1.jpg', 'imagen2.jpg']}


### Comparación de la información

In [23]:
print("Datos originales:")
print(info_original)
print()

print("Datos transmitidos:")
print(info_transmitida)
print()

print("Datos con ruido:")
print(datos_canal)
print()

print("Datos receptados :")
print(info_receptada)

Datos originales:
{'Se detectó': 'roca, cristal, agua & hielo', 'Ubicación': (-87.65, 762.56, -511.55), 'Imagenes': ['imagen1.jpg', 'imagen2.jpg']}

Datos transmitidos:
{'Se detectó': '010010110101100001010010010100010011101000110000010100100100101101011101010010100100111001010001010110100011101000110000010100010101010001001111010100010011000000110101001100000101110001011101010101110101101001011000', 'Ubicación': '00111100001110110010010000101100001110010010110100101111001110100011000000101100001011010010101100111001001011110010110100111010001100000011101100101111001010010010100100111001001011110010111100111101', 'Imagenes': ['imagen1.jpg', 'imagen2.jpg']}

Datos con ruido:
{'Se detectó': '010010110101100001010010010100010011101000110000010100100100101101011101010010100100111001010001010110100011101000110000010100010101010001001111010100010011000000110101001100000101110001011101010101110101101001011000', 'Ubicación': '00111100001110110010010000101100001110010010110100101111001110100011